# Constructing B-cells gene-gene correlation network (GCN) using CS-CORE

This notebook matches this original [Tutorial](https://github.com/ChangSuBiostats/CS-CORE_python/blob/master/analysis/CSCORE_python_example.ipynb)

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
## Core packages
import warnings

warnings.filterwarnings("ignore")

# Numerical
import numpy as np
import pandas as pd

pd.set_option("display.max_columns", None)

# Plotting
import seaborn as sns

%matplotlib inline

sns.set_style("ticks")
%config InlineBackend.figure_format = 'retina'

## Single-cell packages for preprocessing and embedding
import anndata as ad
import scanpy as sc

sc.settings.verbosity = 1  # verbosity: errors (0), warnings (1), info (2), hints (3)
sc.settings.set_figure_params(dpi=150, fontsize=10, dpi_save=150, figsize=(5, 5), format="png")
sc.settings.figdir = "."

from scmetric.external import CSCORE

# Load the filtered B-cells dataset 

In [3]:
adata = ad.read_h5ad("Bcells_dataset.h5ad")

adata

AnnData object with n_obs × n_vars = 1994 × 5000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.rps', 'percent.rpl', 'percent.rrna', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.1', 'seurat_clusters', 'singler', 'Admission.level', 'cell.type.fine', 'cell.type.coarse', 'cell.type', 'IFN1', 'HLA1', 'Donor.orig', 'Donor.full', 'Donor', 'Status', 'Sex', 'DPS', 'DTF', 'Admission', 'Ventilated'
    var: 'features', 'SCT_features'
    obsm: 'X_umap'

# Compute gene-gene correlations using CS-CORE

In [12]:
adata_with_GCN = CSCORE(adata=adata, copy=True)


adata_with_GCN

Iteration 0
Iteration 1
Iteration 2
Iteration 3
IRLS converged after 4 iterations (delta=5.32e-03).


AnnData object with n_obs × n_vars = 1994 × 5000
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.rps', 'percent.rpl', 'percent.rrna', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.1', 'seurat_clusters', 'singler', 'Admission.level', 'cell.type.fine', 'cell.type.coarse', 'cell.type', 'IFN1', 'HLA1', 'Donor.orig', 'Donor.full', 'Donor', 'Status', 'Sex', 'DPS', 'DTF', 'Admission', 'Ventilated', 'seq_depth'
    var: 'features', 'SCT_features', 'mu', 'sigma2', 'has_low_variance'
    obsm: 'X_umap'
    varp: 'corr_mat', 'corr_mat_z', 'corr_mat_pval'

## Inspecting the correlation among the top-3 genes

In [13]:
adata_with_GCN.varp["corr_mat"][0:3, 0:3]

array([[ 1.        ,  0.61152648, -0.04710932],
       [ 0.61152648,  1.        , -0.00449181],
       [-0.04710932, -0.00449181,  1.        ]])

## Mean and variance of expression rates

In [16]:
adata_with_GCN.var.loc[:, ["mu", "sigma2"]]

,mu,sigma2
RNA18S5,0.111836,3.687804e-03
RNA28S5,0.049710,1.037663e-03
MT-RNR2,0.058125,6.154636e-04
IGHM,0.004089,3.410607e-04
MALAT1,0.019931,1.212203e-04
...,...,...
RUNDC1,0.000041,6.726111e-09
LONRF1,0.000035,2.422775e-09
RARS2,0.000030,2.432391e-09
SNAPC3,0.000033,1.801468e-09


# Construct and cluster GCN

In [26]:
genes_adata = adata_with_GCN.T.copy()

beta_unsigned = 6
pval_threshold = 1e-5
C = genes_adata.obsp["corr_mat"]
C[pval_threshold < genes_adata.obsp["corr_mat_pval"]] = 0
np.fill_diagonal(C, 0)

adj = np.abs(C) ** beta_unsigned
genes_adata.obsp["gcn"] = adj

sc.tl.leiden(genes_adata, adjacency=adj, resolution=1.0, key_added="leiden")

genes_adata

AnnData object with n_obs × n_vars = 5000 × 1994
    obs: 'features', 'SCT_features', 'mu', 'sigma2', 'has_low_variance', 'leiden'
    var: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'percent.rps', 'percent.rpl', 'percent.rrna', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.1', 'seurat_clusters', 'singler', 'Admission.level', 'cell.type.fine', 'cell.type.coarse', 'cell.type', 'IFN1', 'HLA1', 'Donor.orig', 'Donor.full', 'Donor', 'Status', 'Sex', 'DPS', 'DTF', 'Admission', 'Ventilated', 'seq_depth'
    uns: 'leiden'
    varm: 'X_umap'
    obsp: 'corr_mat', 'corr_mat_z', 'corr_mat_pval', 'gcn'

## Show the genes in the largest clusters (HLA cluster)

In [35]:
cluster_1 = genes_adata.obs_names[genes_adata.obs["leiden"] == "0"]

cluster_1

Index(['B2M', 'HLA-B', 'HLA-E', 'HLA-A', 'HLA-C', 'PTPRC', 'UBC', 'LCP1',
       'RNF213', 'HSP90AA1',
       ...
       'C11orf21', 'GRAP2', 'CTSW', 'TC2N', 'PLK1S1', 'CHMP5', 'SLC7A6',
       'CBR1', 'PHACTR2', 'KLF11'],
      dtype='object', length=571)